In [ ]:
!pip install LibRecommender

In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
import numpy as np
import pandas as pd
from libreco.data import random_split, split_by_ratio_chrono, DatasetPure
from libreco.algorithms import SVDpp, RNN4Rec  # pure data, algorithm SVD++
from libreco.evaluation import evaluate

In [ ]:
data = pd.read_csv("https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv")
data.columns = ["user", "item", "label", "time"]
# data = pd.read_csv(MOVIE_PATH, sep="::", names=["user", "item", "label", "time"])

# split whole data into three folds for training, evaluating and testing
train_data, eval_data, test_data = split_by_ratio_chrono(data, multi_ratios=[0.8, 0.1, 0.1])

train_data, data_info = DatasetPure.build_trainset(train_data)
eval_data = DatasetPure.build_evalset(eval_data)
test_data = DatasetPure.build_testset(test_data)
print(data_info)   # n_users: 5894, n_items: 3253, data sparsity: 0.4172 %

svdpp = SVDpp(task="rating", data_info=data_info, embed_size=16, n_epochs=3, lr=0.001,
              reg=None, batch_size=256)
# monitor metrics on eval_data during training
svdpp.fit(train_data, verbose=2, eval_data=eval_data, metrics=["rmse", "mae", "r2"])

# do final evaluation on test data
print("evaluate_result: ", evaluate(model=svdpp, data=test_data,
                                    metrics=["rmse", "mae"]))
# predict preference of user 2211 to item 110
print("prediction: ", svdpp.predict(user=2211, item=110))
# recommend 7 items for user 2211
print("recommendation: ", svdpp.recommend_user(user=2211, n_rec=7))

# cold-start prediction
print("cold prediction: ", svdpp.predict(user="ccc", item="not item",
                                         cold_start="average"))
# cold-start recommendation
print("cold recommendation: ", svdpp.recommend_user(user="are we good?",
                                                    n_rec=7,
                                                    cold_start="popular"))


n_users: 610, n_items: 8237, data sparsity: 1.6056 %
Training start time: 2022-10-30 19:46:32


train: 100%|██████████| 316/316 [00:08<00:00, 37.14it/s]


Epoch 1 elapsed: 8.557s
	 train_loss: 6.7125


eval_pred: 100%|██████████| 2/2 [00:00<00:00, 280.64it/s]


	 eval rmse: 1.6842
	 eval mae: 1.3649
	 eval r2: -1.5746


...

Epoch 3 elapsed: 4.284s
	 train_loss: 1.259


eval_pred: 100%|██████████| 2/2 [00:00<00:00, 559.13it/s]


	 eval rmse: 1.1060
	 eval mae: 0.8574
	 eval r2: -0.1104


eval_pred: 100%|██████████| 2/2 [00:00<00:00, 457.47it/s]

evaluate_result:  {'rmse': 1.1385330277090324, 'mae': 0.887031397593311}
Detect 1 unknown interaction(s), position: [0]
prediction:  [3.4911852]
detect unknown user: 2211
recommendation:  [(318, 3.8710794), (593, 3.7615657), (898, 3.7542315), (750, 3.7521029), (1683, 3.7368088), (296, 3.728684), (1041, 3.7017095)]
Detect 1 unknown interaction(s), position: [0]
cold prediction:  [2.0983768]
detect unknown user: are we good?
cold recommendation:  [356, 318, 296, 2571, 593, 260, 110]
